[View in Colaboratory](https://colab.research.google.com/github/DJCordhose/ai/blob/master/notebooks/nlp/2-lstm.ipynb)

In [0]:
# Based on
# https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/6.2-understanding-recurrent-neural-networks.ipynb

In [0]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
print(tf.__version__)

1.11.0-rc2


In [5]:
# let's see what compute devices we have available, hopefully a GPU 
sess = tf.Session()
devices = sess.list_devices()
for d in devices:
    print(d.name)

/job:localhost/replica:0/task:0/device:CPU:0
/job:localhost/replica:0/task:0/device:GPU:0


In [6]:
# a small sanity check, does tf seem to work ok?
hello = tf.constant('Hello TF!')
print(sess.run(hello))

b'Hello TF!'


In [7]:
from tensorflow import keras
print(keras.__version__)

2.1.6-tf


In [0]:
# https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification
max_features = 10000  # number of words to consider as features
maxlen = 500  # cut texts after this number of words (among top max_features most common words)

# each review is encoded as a sequence of word indexes
# indexed by overall frequency in the dataset
# output is 0 (negative) or 1 (positive) 
imdb = tf.keras.datasets.imdb.load_data(num_words=max_features)
(raw_input_train, y_train), (raw_input_test, y_test) = imdb

In [0]:
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences

input_train = tf.keras.preprocessing.sequence.pad_sequences(raw_input_train, maxlen=maxlen)
input_test = tf.keras.preprocessing.sequence.pad_sequences(raw_input_test, maxlen=maxlen)

In [10]:
input_train.shape, input_test.shape, y_train.shape, y_test.shape

((25000, 500), (25000, 500), (25000,), (25000,))

## LSTMs

In [11]:
embedding_dim = 32
dropout = 0.15
recurrent_dropout = 0.2

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(name='embedding', input_dim=max_features, output_dim=embedding_dim, input_length=maxlen))

model.add(tf.keras.layers.LSTM(name='lstm', units=32))

# binary classifier
model.add(tf.keras.layers.Dense(name='fc', units=32, activation='relu'))
model.add(tf.keras.layers.Dense(name='classifier', units=1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 32)           320000    
_________________________________________________________________
lstm (LSTM)                  (None, 32)                8320      
_________________________________________________________________
fc (Dense)                   (None, 32)                1056      
_________________________________________________________________
classifier (Dense)           (None, 1)                 33        
Total params: 329,409
Trainable params: 329,409
Non-trainable params: 0
_________________________________________________________________


In [12]:
batch_size = 1000

%time history = model.fit(input_train, y_train, epochs=10, batch_size=batch_size, validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 34s 2ms/step - loss: 0.6913 - acc: 0.5514 - val_loss: 0.6871 - val_acc: 0.5742
Epoch 2/10
20000/20000 [==============================] - 33s 2ms/step - loss: 0.6509 - acc: 0.6949 - val_loss: 0.5333 - val_acc: 0.7658
Epoch 3/10
20000/20000 [==============================] - 33s 2ms/step - loss: 0.5085 - acc: 0.7817 - val_loss: 0.4583 - val_acc: 0.7912
Epoch 4/10
20000/20000 [==============================] - 33s 2ms/step - loss: 0.3579 - acc: 0.8669 - val_loss: 0.3470 - val_acc: 0.8536
Epoch 5/10
20000/20000 [==============================] - 33s 2ms/step - loss: 0.2510 - acc: 0.9040 - val_loss: 0.3053 - val_acc: 0.8748
Epoch 6/10
20000/20000 [==============================] - 33s 2ms/step - loss: 0.1916 - acc: 0.9325 - val_loss: 0.2956 - val_acc: 0.8812
Epoch 7/10
20000/20000 [==============================] - 33s 2ms/step - loss: 0.1552 - acc: 0.9483 - val_loss: 0.2969 - val_acc:

In [13]:
train_loss, train_accuracy = model.evaluate(input_train, y_train, batch_size=batch_size)
train_accuracy

25000/25000 [==============================] - 14s 568us/step


0.948560004234314

In [14]:
test_loss, test_accuracy = model.evaluate(input_test, y_test, batch_size=batch_size)
test_accuracy

25000/25000 [==============================] - 14s 570us/step


0.8644800066947937

In [15]:
# precition
model.predict(input_test[0:5])

array([[0.1685272 ],
       [0.9979431 ],
       [0.8867608 ],
       [0.32337418],
       [0.9989875 ]], dtype=float32)

In [16]:
# ground truth
y_test[0:5]

array([0, 1, 1, 0, 1])